In [5]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

In [11]:
import xarray as xr

filename = "OBS6_ERA5_reanaly_1_day_pr_2000-2018.nc"  # Update the path to the correct location of the file
# Read the NetCDF file
ds = xr.open_dataset(filename)
# Convert to a pandas DataFrame
df = ds.to_dataframe()
# Reset the index if needed
df.reset_index(inplace=True)
df = df[df['bnds'] == 0]
df.index = pd.to_datetime(df['time'])
df.index = df.index.normalize()
df.drop(columns=['time', 'bnds'], inplace=True)
prec  = df['pr']  # Precipitation data
prec  = prec*86400  # Convert from m/s to mm/day)
prec[prec < 0] = 0  # Set negative values to zero
prec = prec[(prec.index >= '2000-01-01') & (prec.index <= '2017-09-30')]  # Filter the DataFrame for the desired date range


filename = "Derived_Makkink_evspsblpot_2000_2018.nc"  # Update the path to the correct location of the file
# Read the NetCDF file
dsEP = xr.open_dataset(filename)
# Convert to a pandas DataFrame
dfEP = dsEP.to_dataframe()
# Reset the index if needed
dfEP.reset_index(inplace=True)
EP = dfEP['evspsblpot']  # Evapotranspiration data
EP.index = pd.to_datetime(dfEP['time'])
EP.index = EP.index.normalize()
EP = EP*86400  # Convert from mm/s to mm/day)
EP[EP < 0] = 0  # Set negative values to zero
EP = EP[(EP.index >= '2000-01-01') & (EP.index <= '2017-09-30')] 

filename = "6227510_Q_Day.Cmd.txt"
A = 21497*10**6 #m^2
dfQ = pd.read_csv(filename, delimiter=";", skiprows=37, header=None, encoding='latin1')  # Specify encoding
dfQ = dfQ[[0, 2]]  # Select only the first and third columns
dfQ.columns = ['Date', 'Q']  # Rename columns for clarity
dfQ['Date'] = pd.to_datetime(dfQ['Date'], format='%Y-%m-%d')  # Convert 'Date' column to datetime
dfQ.set_index('Date', inplace=True)  # Set 'Date' as the index
dfQ['Q'] = (dfQ['Q']/A)*(10**3)*86400 #mm/day
dfQ = dfQ[(dfQ.index >= '2000-01-01') & (dfQ.index <= '2017-09-30')]  # Filter the DataFrame for the desired date range


In [12]:
n = len(prec)
t = np.arange(n)
Si1 = np.zeros(n)
Si2 = np.zeros(n)
Si_max = 2.5 #?????????????????????????????????????????????????????????????????''
Pe_1 = np.zeros(n)
Pe_2 = np.zeros(n)

for t in np.arange(1,n,1):

    Si1[t] = Si1[t-1] + prec.iloc[t]  # Use .iloc to get the scalar value
    if Si1[t] > Si_max:
        Pe_1[t] = Si1[t] - Si_max
    else:
        Pe_1[t] = 0
    Si1[t] = Si1[t] - Pe_1[t]
    deltS = Si1[t] - EP.iloc[t]  # Use .iloc to get the scalar value
    Si1[t] = np.maximum(0, deltS)

Pe_1_mean = Pe_1.mean()


print(Pe_1_mean/prec.mean())


0.5881277355398693


In [13]:
print(len(prec.index))
print(len(EP.index))
print(len(dfQ.index))
print(len(Si1))
print(len(Pe_1))


6483
6483
6483
6483
6483


In [14]:
data = pd.DataFrame({'P': prec, 'EP': EP, 'Q': dfQ['Q'].values, 'S': Si1, 'Pe': Pe_1}, index=prec.index)
data.head(15)

,P,EP,Q,S,Pe
time,,,,,
2000-01-01,0.004016,1.168847,0.034243,0.000000,0.000000
2000-01-02,0.000000,1.184061,0.034726,0.000000,0.000000
2000-01-03,0.020909,1.059634,0.034927,0.000000,0.000000
2000-01-04,0.000000,1.175390,0.034525,0.000000,0.000000
2000-01-05,0.000000,1.182336,0.035047,0.000000,0.000000
2000-01-06,0.000793,1.199630,0.035128,0.000000,0.000000
2000-01-07,0.000000,1.223502,0.035972,0.000000,0.000000
2000-01-08,0.000000,0.953342,0.036414,0.000000,0.000000
2000-01-09,1.723231,1.049964,0.037338,0.673267,0.000000


In [15]:
P = data['P'].mean()
Pe = data['Pe'].mean()

print(Pe/prec.mean())

0.5881277355398693


In [16]:
#PROBLEM 2
P_mean = data['P'].mean()
EP_mean = data['EP'].mean()
Q_mean = data['Q'].mean()

ET_mean = P_mean - Q_mean #transpiration

#print (P_mean, EP_mean, Q_mean, ET_mean)

data.loc[:,'ET'] = (data['EP']/EP_mean)*ET_mean

data.loc[:,'P-ET'] = data['P'] - data['ET']


for i in range(1, len(data)):
    if data['P-ET'].iloc[:i].cumsum().iloc[-1] > 0:
        data.loc[data.index[i], 'SD'] = 0
    else:
        data.loc[data.index[i], 'SD'] = data['P-ET'].iloc[:i].cumsum().iloc[-1]


SR = data['SD'].min()
        
print(-SR)

296.46462980954635
